In [44]:
#!pip3.9 install pyspark
!pip3.10 install pyspark

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [45]:
!export PYSPARK_PYTHON=python3.10
!export PYSPARK_DRIVER_PYTHON=python3.10

In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
import shutil

# Crear la sesión Spark
spark = SparkSession.builder.appName("keywordCount").getOrCreate()

# Leer el archivo CSV
csv_path = "general.csv"
csv_df = spark.read.csv(csv_path, header=True, inferSchema=True)

# Filtrar registros con columnas no nulas
filtered_df = csv_df.filter(col('abstract').isNotNull())

# MAP-REDUCE
# Seleccionar el campo abstract y convertirlo en un RDD
words_rdd = filtered_df.select("abstract").rdd.flatMap(lambda x: x[0].split())

# Lista de palabras
words_to_count = ["Data", "Article", "a", "The"]
words_to_count = [word.lower() for word in words_to_count]

# Contar las palabras
counts_rdd = words_rdd.filter(lambda word: word.lower() in words_to_count).map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)

# Mostrar los resultados
print(counts_rdd.collect())

# Guardar los resultados en CSV
counts_df = counts_rdd.toDF(["word", "frequency"])  # convertir a DataFrame la lista resultado
counts_df.write.csv("Keywords", header=True)

# Cambiamos el nombre del csv generado a "Keywords.csv" y lo extraemos del folder
for file_name in os.listdir('Keywords'):
    if file_name.endswith('.csv'):
        current_path = os.path.join('Keywords', file_name)
        new_path = os.path.join('Keywords', 'Keywords.csv')
        os.rename(current_path, new_path)
        break
else:
    print("No CSV file found in the 'Keywords' folder.")

# Extraemos el csv y eliminamos el folder
folder = os.path.abspath('Keywords')
file_path = os.path.join(folder, 'Keywords.csv')
new_path = os.path.join(os.path.dirname(folder), 'Keywords.csv')
shutil.move(file_path, new_path)
shutil.rmtree('Keywords')

# Cerrar la sesión de Spark
spark.stop()


[('a', 8), ('the', 35), ('data', 13)]
